# EDA Data Collection and Processing file

By Aommy, Austin, Ling, Tath (yippee)

-----------------------

## Installing Dependencies

### Minio Object Store dependencies

1. Install minio cli if not already installed.
2. Create client
3. Make bucket (if needed)

# no longer needed!

In [1]:
! pip install minio --quiet

^C
ERROR: Operation cancelled by user


In [1]:
from minio import Minio

# Create client with access and secret key.
# client = Minio("s3.amazonaws.com", "ACCESS-KEY", "SECRET-KEY")

# Create client with access key and secret key with specific region.
client = Minio(
    "localhost:9000",
    access_key="ROOTNAME",
    secret_key="CHANGEME123",
    secure=False
)

if client.bucket_exists("eda"):
    print("Bucket exists.")
else:
    client.make_bucket("eda")
    print("Bucket created.")

Bucket exists.


#### DuckDB dependencies

In [2]:
!pip uninstall --quiet --yes malloy
!pip install --quiet --upgrade duckdb
!pip install --quiet jupysql==0.10.12
!pip install --quiet duckdb-engine

## Importing libraries

In [46]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import yfinance as yf



import io
from urllib.request import urlopen
import zipfile
import os

In [3]:
!pip install --quiet yfinance --upgrade --no-cache-dir

---------------------

## Data Collection

### Data from Global Events

Installing data from http://data.gdeltproject.org/events/index.html for global event data!

In [48]:
for y in range(19, 25):
    for m in range(1, 13):
        if m < 10:
            m = "0" + str(m)
        for d in range(1, 32):
            if d < 10:
                d = "0" + str(d)
            try:
                response = client.get_object("eda", "gdelt-parquet/20{0}/{1}/{2}.export.CSV.zip".format(y,m,d))
                # Read data from response.
            except:
                print("File doesn't exist!")
                try:
                    data = urlopen(
                        "http://data.gdeltproject.org/events/20{0}{1}{2}.export.CSV.zip".format(y,m,d),
                    )
                    print("Uploading file!")
                    result = client.put_object(
                        "eda", "gdelt-parquet/20{0}/{1}/{2}.export.CSV.zip".format(y,m,d), data, length=-1, part_size=10*1024*1024,
                    )
                    print(
                        "created {0} object; etag: {1}, version-id: {2}".format(
                            result.object_name, result.etag, result.version_id,
                        ),
                    )
                except:
                    print("No file to download!")
            finally:
                response.close()
                response.release_conn()

In [4]:
gdelt_headers = {
    "GlobalEventID": int,
    "Day": int,
    "MonthYear": int,
    "Year": int,
    "FractionDate": float,
    "Actor1Code": str,
    "Actor1Name": str,
    "Actor1CountryCode": str,
    "Actor1KnownGroupCode": str,
    "Actor1EthnicCode": str,
    "Actor1Religion1Code": str,
    "Actor1Religion2Code": str,
    "Actor1Type1Code": str,
    "Actor1Type2Code": str,
    "Actor1Type3Code": str,
    "Actor2Code": str,
    "Actor2Name": str,
    "Actor2CountryCode": str,
    "Actor2KnownGroupCode": str,
    "Actor2EthnicCode": str,
    "Actor2Religion1Code": str,
    "Actor2Religion2Code": str,
    "Actor2Type1Code": str,
    "Actor2Type2Code": str,
    "Actor2Type3Code": str,
    "IsRootEvent": bool,
    "EventCode": str,
    "EventBaseCode": str,
    "EventRootCode": str,
    "QuadClass": float,
    "GoldsteinScale": float,
    "NumMentions": float,
    "NumSources": float,
    "NumArticles": float,
    "AvgTone": float,
    "Actor1Geo_Type": float,
    "Actor1Geo_Fullname": str,
    "Actor1Geo_CountryCode": str,
    "Actor1Geo_ADM1Code": str,
    "Actor1Geo_Lat": float,
    "Actor1Geo_Long": object,
    "Actor1Geo_FeatureID": object,
    "Actor2Geo_Type": float,
    "Actor2Geo_Fullname": str,
    "Actor2Geo_CountryCode": str,
    "Actor2Geo_ADM1Code": str,
    "Actor2Geo_Lat": float,
    "Actor2Geo_Long": object,
    "Actor2Geo_FeatureID": object,
    "ActionGeo_Type": float,
    "ActionGeo_Fullname": str,
    "ActionGeo_CountryCode": str,
    "ActionGeo_ADM1Code": str,
    "ActionGeo_Lat": float,
    "ActionGeo_Long": object,
    "ActionGeo_FeatureID": object,
    "DateAdded": float,
    "SOURCEURL": str
}

In [5]:
len(gdelt_headers)

58

In [6]:
gdelt_headers.keys()

dict_keys(['GlobalEventID', 'Day', 'MonthYear', 'Year', 'FractionDate', 'Actor1Code', 'Actor1Name', 'Actor1CountryCode', 'Actor1KnownGroupCode', 'Actor1EthnicCode', 'Actor1Religion1Code', 'Actor1Religion2Code', 'Actor1Type1Code', 'Actor1Type2Code', 'Actor1Type3Code', 'Actor2Code', 'Actor2Name', 'Actor2CountryCode', 'Actor2KnownGroupCode', 'Actor2EthnicCode', 'Actor2Religion1Code', 'Actor2Religion2Code', 'Actor2Type1Code', 'Actor2Type2Code', 'Actor2Type3Code', 'IsRootEvent', 'EventCode', 'EventBaseCode', 'EventRootCode', 'QuadClass', 'GoldsteinScale', 'NumMentions', 'NumSources', 'NumArticles', 'AvgTone', 'Actor1Geo_Type', 'Actor1Geo_Fullname', 'Actor1Geo_CountryCode', 'Actor1Geo_ADM1Code', 'Actor1Geo_Lat', 'Actor1Geo_Long', 'Actor1Geo_FeatureID', 'Actor2Geo_Type', 'Actor2Geo_Fullname', 'Actor2Geo_CountryCode', 'Actor2Geo_ADM1Code', 'Actor2Geo_Lat', 'Actor2Geo_Long', 'Actor2Geo_FeatureID', 'ActionGeo_Type', 'ActionGeo_Fullname', 'ActionGeo_CountryCode', 'ActionGeo_ADM1Code', 'ActionGeo_

### Data from YahooFinance

In [7]:
import yfinance as yf

companies = {
    "tech": ['AAPL', 'MSFT', 'NVDA'],
    "energy": ['XOM', 'CVX', 'BP', 'COP'],
    "food": ['KO', 'PEP', 'PG'],
    "health": ['JNJ', 'PFE', 'ABBV'],
    "finance": ['JPM', 'V', 'MA'],
    "agriculture": ['DE', 'CTVA', 'ADM'],
    "telecom": ['T', 'VZ', 'TMUS'],
    "materials": ['DOW', 'NEM', 'FCX']
}

In [53]:
uploaded = False
for sector in companies:
    for stock in companies[sector]:
        try:
            response = client.get_object("eda", "yfinance/{0}/{1}.snappy.parquet".format(sector,stock))
            # Read data from response.
        except:
            print("File doesn't exist!")
            data = yf.download(stock, start="2019-01-01", end="2024-10-15", progress=False)
            result = client.put_object(
                "eda", "yfinance/{0}/{1}.snappy.parquet".format(sector,stock), io.BytesIO(data.to_parquet(compression='snappy')), length=-1, part_size=10*1024*1024,
            )
            print(
                "created {0} object; etag: {1}, version-id: {2}".format(
                    result.object_name, result.etag, result.version_id,
                ),
            )
            uploaded = True
        finally:
            response.close()
            response.release_conn()
if (not uploaded):
    print("No changes to make.")

----------------------

## Data Processing

Convering .CSV.zip to .snappy.parquet!

In [57]:
for y in range(19, 25):
    for m in range(1, 13):
        if m < 10:
            m = "0" + str(m)
        for d in range(1, 32):
            if d < 10:
                d = "0" + str(d)
            try:
                response = client.get_object("eda", "gdelt-parquet/20{0}/{1}/{2}.snappy.parquet".format(y,m,d))
                # Read data from response.
            except:
                print("File doesn't exist! 20{0}/{1}/{2}".format(y,m,d))
                try:
                    response = client.get_object("eda", "gdelt/20{0}/{1}/{2}.export.CSV.zip".format(y,m,d))
                    client.fget_object("eda", "gdelt/20{0}/{1}/{2}.export.CSV.zip".format(y,m,d), 
                                       "./data/gdelt/20{0}{1}{2}.export.CSV.zip".format(y,m,d))
                    with zipfile.ZipFile("./data/gdelt/20{0}{1}{2}.export.CSV.zip".format(y,m,d),"r") as zip_ref:
                        zip_ref.extractall("./data/gdelt/temp")
                    data = pd.read_csv("./data/gdelt/temp/20{0}{1}{2}.export.CSV".format(y,m,d), sep='\t', header=None, names=gdelt_headers.keys(), dtype=gdelt_headers, on_bad_lines='warn')
                    result = client.put_object(
                        "eda", "gdelt-parquet/20{0}/{1}/{2}.snappy.parquet".format(y,m,d), io.BytesIO(data.to_parquet(compression='snappy')), length=-1, part_size=10*1024*1024,
                    )
                    print(
                        "created {0} object; etag: {1}, version-id: {2}".format(
                            result.object_name, result.etag, result.version_id,
                        ),
                    )
                    os.remove("./data/gdelt/temp/20{0}{1}{2}.export.CSV".format(y,m,d))
                    os.remove("./data/gdelt/20{0}{1}{2}.export.CSV.zip".format(y,m,d))
                except:
                    print("This date does not exist!!!")

----------------------

## Data Analysis

In [8]:
import duckdb as ddb

In [9]:
! pip install pyarrow --quiet
%load_ext sql

There's a new jupysql version available (0.10.14), you're running 0.10.12. To upgrade: pip install jupysql --upgrade
Deploy FastAPI apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [10]:
%sql ROLLBACK

In [11]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [ ]:
%%sql
INSTALL httpfs;
LOAD httpfs;
SET s3_region='us-east-1';
SET s3_url_style='path';
SET s3_endpoint='localhost:9000';
SET s3_access_key_id='ROOTNAME' ;
SET s3_secret_access_key='CHANGEME123';
SET s3_use_ssl = false;

In [317]:
%sql DROP TABLE gdelt

In [67]:
%sql CREATE TABLE gdelt AS SELECT * FROM read_parquet('s3://eda/gdelt-parquet/2019/*/*.snappy.parquet', union_by_name=True);

In [69]:
# %sql SELECT count(*) FROM gdelt

In [70]:
%sql create table yfinance as select * from read_parquet("s3://eda/yfinance/*/*.snappy.parquet")

In [71]:
# %sql SELECT count(*) FROM yfinance

In [72]:
%sql INSERT INTO gdelt SELECT * FROM read_parquet("s3://eda/gdelt-parquet/2020/*/*.snappy.parquet")

In [73]:
%sql INSERT INTO gdelt SELECT * FROM read_parquet("s3://eda/gdelt-parquet/2021/*/*.snappy.parquet")

In [74]:
%sql INSERT INTO gdelt SELECT * FROM read_parquet("s3://eda/gdelt-parquet/2022/*/*.snappy.parquet")

In [75]:
%sql INSERT INTO gdelt SELECT * FROM read_parquet("s3://eda/gdelt-parquet/2023/*/*.snappy.parquet")

In [76]:
%sql INSERT INTO gdelt SELECT * FROM read_parquet("s3://eda/gdelt-parquet/2024/*/*.snappy.parquet")

In [41]:
%sql SELECT count(*) from gdelt

,count_star()
0,260658282


In [14]:
%sql SELECT * FROM gdelt LIMIT 3;

,GlobalEventID,Day,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor2Code,...,NumSources,NumArticles,AvgTone,Actor1Geo_Type,Actor1Geo_CountryCode,Actor2Geo_Type,Actor2Geo_CountryCode,ActionGeo_Type,ActionGeo_CountryCode,SOURCEURL
0,813415774,20180101,201801,2018,2018.0027,None,None,None,None,AGR,...,6.0,45.0,-1.070367,0.0,None,4.0,CA,4.0,CA,https://www.weyburnreview.com/cn-rail-and-cp-r...
1,813415775,20180101,201801,2018,2018.0027,None,None,None,None,USA,...,1.0,8.0,-1.333333,0.0,None,3.0,US,3.0,US,http://www.startribune.com/arcelormittal-takes...
2,813415776,20180101,201801,2018,2018.0027,AGR,FARMER,None,None,None,...,3.0,15.0,-1.112403,4.0,CA,0.0,None,4.0,CA,https://www.weyburnreview.com/cn-rail-and-cp-r...


In [1]:
%%sql
ALTER TABLE gdelt DROP COLUMN DateAdded;
ALTER TABLE gdelt DROP COLUMN ActionGeo_Lat;
ALTER TABLE gdelt DROP COLUMN ActionGeo_Long;
ALTER TABLE gdelt DROP COLUMN ActionGeo_FeatureID;
ALTER TABLE gdelt DROP COLUMN ActionGeo_ADM1Code;
ALTER TABLE gdelt DROP COLUMN ActionGeo_Fullname;
ALTER TABLE gdelt DROP COLUMN Actor2Geo_ADM1Code;
ALTER TABLE gdelt DROP COLUMN Actor2Geo_Lat;
ALTER TABLE gdelt DROP COLUMN Actor2Geo_Long;
ALTER TABLE gdelt DROP COLUMN Actor2Geo_FeatureID;
ALTER TABLE gdelt DROP COLUMN Actor2Geo_Fullname;
ALTER TABLE gdelt DROP COLUMN Actor1Geo_ADM1Code;
ALTER TABLE gdelt DROP COLUMN Actor1Geo_Lat;
ALTER TABLE gdelt DROP COLUMN Actor1Geo_Long;
ALTER TABLE gdelt DROP COLUMN Actor1Geo_FeatureID;
ALTER TABLE gdelt DROP COLUMN Actor1Geo_Fullname;
ALTER TABLE gdelt DROP COLUMN Actor1EthnicCode;
ALTER TABLE gdelt DROP COLUMN Actor2EthnicCode;
ALTER TABLE gdelt DROP COLUMN Actor1Religion1Code;
ALTER TABLE gdelt DROP COLUMN Actor1Religion2Code;
ALTER TABLE gdelt DROP COLUMN Actor2Religion1Code;
ALTER TABLE gdelt DROP COLUMN Actor2Religion2Code;
ALTER TABLE gdelt DROP COLUMN Actor1Type1Code;
ALTER TABLE gdelt DROP COLUMN Actor1Type2Code;
ALTER TABLE gdelt DROP COLUMN Actor1Type3Code;
ALTER TABLE gdelt DROP COLUMN Actor2Type1Code;
ALTER TABLE gdelt DROP COLUMN Actor2Type2Code;
ALTER TABLE gdelt DROP COLUMN Actor2Type3Code;

UsageError: Cell magic `%%sql` not found.


In [72]:
%sql SELECT * FROM gdelt LIMIT 3;

,GlobalEventID,Day,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor2Code,...,NumSources,NumArticles,AvgTone,Actor1Geo_Type,Actor1Geo_CountryCode,Actor2Geo_Type,Actor2Geo_CountryCode,ActionGeo_Type,ActionGeo_CountryCode,SOURCEURL
0,813415774,20180101,201801,2018,2018.0027,None,None,None,None,AGR,...,6.0,45.0,-1.070367,0.0,None,4.0,CA,4.0,CA,https://www.weyburnreview.com/cn-rail-and-cp-r...
1,813415775,20180101,201801,2018,2018.0027,None,None,None,None,USA,...,1.0,8.0,-1.333333,0.0,None,3.0,US,3.0,US,http://www.startribune.com/arcelormittal-takes...
2,813415776,20180101,201801,2018,2018.0027,AGR,FARMER,None,None,None,...,3.0,15.0,-1.112403,4.0,CA,0.0,None,4.0,CA,https://www.weyburnreview.com/cn-rail-and-cp-r...


In [81]:
%%sql
select *
from INFORMATION_SCHEMA.COLUMNS
where TABLE_NAME='gdelt'

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable,COLUMN_COMMENT
0,eda-gdelt,main,gdelt,GlobalEventID,1,None,YES,BIGINT,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
1,eda-gdelt,main,gdelt,Day,2,None,YES,BIGINT,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
2,eda-gdelt,main,gdelt,MonthYear,3,None,YES,BIGINT,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
3,eda-gdelt,main,gdelt,Year,4,None,YES,BIGINT,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
4,eda-gdelt,main,gdelt,FractionDate,5,None,YES,DOUBLE,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
5,eda-gdelt,main,gdelt,Actor1Code,6,None,YES,VARCHAR,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
6,eda-gdelt,main,gdelt,Actor1Name,7,None,YES,VARCHAR,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
7,eda-gdelt,main,gdelt,Actor1CountryCode,8,None,YES,VARCHAR,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
8,eda-gdelt,main,gdelt,Actor1KnownGroupCode,9,None,YES,VARCHAR,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
9,eda-gdelt,main,gdelt,Actor2Code,10,None,YES,VARCHAR,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
